<a href="https://colab.research.google.com/github/monilchheda/manning-live-project-building-domain-specific-language-models/blob/master/building_domain_specific_language_models_1_preparing_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from nltk.tokenize import WordPunctTokenizer

tk = WordPunctTokenizer() 

In [0]:
def clean_it(txt):
    #print(type(txt))
    # I am sure there is a better way to handle this, but, here goes nothing :-D

    #need to convert the input (which is list type) to string, by adding ''
    original = ''.join(txt)
    
    # remove extra $$ - could not figure out a way to remove with regx below for latex
    no_multiple_dollar = original.replace("$$","$")
    
    # BeautifulSoup to clean up majority html tags
    bs_clean = BeautifulSoup(no_multiple_dollar, 'html.parser').get_text()
    
    # Stray links need to be removed
    url_pattern = r'https?:\/\/.*'
    url_pattern_cleaned = re.sub(url_pattern , "", bs_clean)
    
    # remove most latex patterns, some strays like postid 239921 not being cleaned... strange
    latex_pattern = r'(?<!\\)\$.*?(?<!\\)\$'
    latex_pattern_cleaned = re.sub(latex_pattern , "", url_pattern_cleaned)
    
    # remove carriage returns - no suitable example found, in the blind here
    carriage_return_pattern = r'(?<![\r\n])(?:\r\n|\r|\n){2}[^\r\n]+'
    carriage_return_cleaned = re.sub(carriage_return_pattern , "", latex_pattern_cleaned)
    
    # remove all digits - 240019
    number_pattern = r'[0-9]+'
    number_pattern_cleaned = re.sub(number_pattern , "", carriage_return_cleaned)
    
    # remove usernames/ twitter handles - 240019
    user_quote_pattern = r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)'
    user_quote_cleaned = re.sub(user_quote_pattern , "", number_pattern_cleaned)
    

    # brackets
    #    brackets_pattern = r' ?\([^)]+\)'
    round_brackets_pattern = r'[\(\)]'
    round_brackets_pattern_clean = re.sub(round_brackets_pattern , "", user_quote_cleaned)
    
    # square brackets
    square_brackets_pattern = r'[\[\]]'
    square_brackets_pattern_clean = re.sub(square_brackets_pattern, "", round_brackets_pattern_clean)
    
    # Curly brackets
    curly_brackets_pattern = r'[\{\}]'
    curly_brackets_pattern_clean = re.sub(curly_brackets_pattern, "", square_brackets_pattern_clean)
    
    # Asterisk
    asterisk_brackets_pattern = r'[\*]'
    asterisk_brackets_pattern_clean = re.sub(asterisk_brackets_pattern, "", curly_brackets_pattern_clean)

    # remove almost anything not a word? - Maybe too strong.. https://stackoverflow.com/questions/875968/how-to-remove-symbols-from-a-string-with-python 
    #misc_cleanup_pattern = r'[^\w]'
    #misc_pattern_cleaned = re.sub(misc_cleanup_pattern, " ", user_quote_cleaned)
    
    ## No clue for below items:
    # 3. Remove missing values for texts
    # 4. Remove texts that are extremely large or too short to bring any information to the model. We want to keep paragraphs that contain at least a few words and remove the paragraphs that are composed of large numerical tables.
    # emojis etc
    
    
    result = asterisk_brackets_pattern_clean
    
    tok_result = tk.tokenize(result.lower())
    
    return tok_result

In [3]:
!wget https://liveproject-resources.s3.amazonaws.com/116/other/stackexchange_812k.csv.gz
#import os
#print(os.getcwd())

!gunzip stackexchange_812k.csv.gz

df_full = pd.read_csv('stackexchange_812k.csv')
df_full.info()
df_full['cleaned_tokenized'] = df_full['text'].apply(clean_it)
print(df_full.head())
df_full.to_csv('stackexchange_812k_cleaned_tokenized.csv')

--2020-05-18 00:02:59--  https://liveproject-resources.s3.amazonaws.com/116/other/stackexchange_812k.csv.gz
Resolving liveproject-resources.s3.amazonaws.com (liveproject-resources.s3.amazonaws.com)... 52.216.30.52
Connecting to liveproject-resources.s3.amazonaws.com (liveproject-resources.s3.amazonaws.com)|52.216.30.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130230720 (124M) [application/x-gzip]
Saving to: ‘stackexchange_812k.csv.gz’

stackexchange_812k. 100%[===================>] 124.20M  13.7MB/s    in 11s     

2020-05-18 00:03:11 (11.3 MB/s) - ‘stackexchange_812k.csv.gz’ saved [130230720/130230720]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812132 entries, 0 to 812131
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   post_id     812132 non-null  int64  
 1   parent_id   75535 non-null   float64
 2   comment_id  553076 non-null  float64
 3   text        812132 non-null  o

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "https://web.archive.org/web/20100822160653/http://mathforum.org/workshops/sum96/data.collections/datalibrary/data.set6.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://xkcd.com/552/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://meta.stats.stackexchange.com/questions/35/how-much-programming-here" looks like a URL. Beautiful Soup is not an HTTP client. You should prob

   post_id  ...                                  cleaned_tokenized
0        1  ...                 [eliciting, priors, from, experts]
1        2  ...                           [what, is, normality, ?]
2        3  ...  [what, are, some, valuable, statistical, analy...
3        4  ...  [assessing, the, significance, of, differences...
4        6  ...  [the, two, cultures, :, statistics, vs, ., mac...

[5 rows x 6 columns]
